### Hyperparameter tuning with grid search

In [ ]:
import torch
from core.util.hyperparameter_configuration import get_hyperparameter_configuration
from core.util.get_datasets import cross_validation
from core.models.model_training import train_model
from sklearn.model_selection import ParameterGrid
from core.util.save_model import save_parameters, load_parameters
from core.util.trefor_dataset import TreforData
from torch.utils.data import DataLoader
from core.models import LSTM

Set ML model, loss function and hyperparameters that that will be tested.

In [ ]:
experiment_name = "01_LSTM_trefor_park"
features = {}
model_input_size = len(features) + 1
model_used = LSTM

In [ ]:
experiment_parameters = load_parameters(experiment_name)
learning_rate = experiment_parameters["learning_rate"]
batch_size = experiment_parameters["batch_size"]
lookback = experiment_parameters["lookback"]
num_layers = experiment_parameters["num_layers"]
# torch.manual_seed(experiment_parameters["seed"])

experiment_parameters

In [ ]:
hyperparameters = get_hyperparameter_configuration()
hidden_size = hyperparameters["hidden_size"]
epochs = hyperparameters["epochs"]
horizon = hyperparameters["horizon"]
loss_function = hyperparameters["loss_function"]
dropout_rate = hyperparameters["dropout_rate"]
train_days = hyperparameters["train_days"]
val_days = hyperparameters["val_days"]
test_days = hyperparameters["test_days"]
early_stopper = hyperparameters["early_stopper"]

hyperparameters

In [2]:
# Specific to experiment
features = {
    "Month_x": True,
    "Month_y": True,
    "Hour_x": True,
    "Hour_y": True,
    "Day_x": True,
    "Day_y": True,
}

Use CUDA (GPU) if available.

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

Train a model with specified hyperparameters

In [ ]:
def train_with_params(params: dict) -> tuple[float, model_used]:
    """Train model with the specified hyperparameters."""
    # Extract hyperparameters
    batch_size = params["batch_size"]
    learning_rate = params["learning_rate"]
    num_layers = params["num_layers"]
    lookback = params["lookback"]
    torch.manual_seed(params["seed"])

    x_train, y_train, x_val, y_val, x_test, y_test = cross_validation(
        lookback=lookback,
        horizon=horizon,
        train_days=train_days,
        val_days=val_days,
        test_days=test_days,
        features=features,
    )

    # convert to dataset that can use dataloaders
    train_dataset = TreforData(x_train, y_train, device)
    val_dataset = TreforData(x_val, y_val, device)

    # initialize the dataloaders, without shuffeling the data between epochs
    training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = model_used(
        input_size=model_input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout_rate=dropout_rate,
        horizon=horizon,
        lookback=lookback,
    ).to(device)

    _, val_loss, best_model = train_model(
        model=model,
        training_loader=training_loader,
        validation_loader=validation_loader,
        learning_rate=learning_rate,
        early_stopper=early_stopper,
    )

    return min(val_loss), best_model

Iterate over all hyperparameters and train a model for each combination.

In [ ]:
best_loss = float("inf")
best_params = None
best_model = None

for params in ParameterGrid(experiment_parameters):
    early_stopper.reset()
    v_loss, model = train_with_params(params)
    print(params, v_loss)
    if v_loss < best_loss:
        best_loss = v_loss
        best_params = params
        best_model = model

save_parameters(experiment_name, best_params, best_loss)
print("Best Hyperparameters:", best_params)
print("Best Validation Loss:", best_loss)

{'Month_x': True, 'Month_y': True, 'Hour_x': True, 'Hour_y': True, 'Day_x': True, 'Day_y': True}
(12473, 5)
{'Month_x': True, 'Month_y': True, 'Hour_x': True, 'Hour_y': True, 'Day_x': True, 'Day_y': True}
(18853, 5)
{'Month_x': True, 'Month_y': True, 'Hour_x': True, 'Hour_y': True, 'Day_x': True, 'Day_y': True}
(16898, 5)
{'Month_x': True, 'Month_y': True, 'Hour_x': True, 'Hour_y': True, 'Day_x': True, 'Day_y': True}
(18288, 5)
{'Month_x': True, 'Month_y': True, 'Hour_x': True, 'Hour_y': True, 'Day_x': True, 'Day_y': True}
(23352, 5)
{'Month_x': True, 'Month_y': True, 'Hour_x': True, 'Hour_y': True, 'Day_x': True, 'Day_y': True}
(23352, 5)


Iterating epochs:   0%|          | 0/200 [00:00<?, ?it/s]

RuntimeError: input.size(-1) must be equal to input_size. Expected 7, got 5